In [78]:
import csv
import sys
import time
import math

INFINITE = 999999



#Reader로 .csv파일을 입력되는 노드 그래프로 변환함
class CSVReader:
    def __init__(self):
        pass  #input 없는 클래스는 그냥 이렇게 쓴다
    
    def load_csv(self, path):
        nodes = [ ] #노드 리스트 생성

        with open(path, newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',')
            node_idx = 0
            for row in spamreader:
                distances = [int(x) for x in row]
                n = Node(node_idx, distances) #n을 노드로서 속성 부여
                node_idx += 1
                nodes.append(n) #노드 리스트에 노드 번호와 그 노드에서 각 대상 노드들로의 거리들을 넣음
        
        graph_info = GraphInfo(nodes) #그래프 클래스에 완성된 노드 리스트를 넣음
        return graph_info


    
#특정 시점의 그래프의 배치 정보를 출력
class GraphInfo:
    #노드셋 정보들을 받음
    def __init__(self, nodes):
        self.nodes = nodes
    
    #노드셋에서 특정 대상 노드를 반환// getter
    def get_node(self, index):
        return self.nodes[index]
    
    #그래프 내 노드의 개수
    def get_node_cnt(self):
        return len(self.nodes)
    
    #두 노드 사이의 거리
    def get_distance(self, n1, n2):
        return self.get_node(n1).get_distance(n2) #get_distance를 같은 용어로 씀
    
    #그래프 내 모든 노드 방문상태 초기화
    def init_node_visit_status(self):
        for node in self.nodes:
            node.set_visited(False)
                        
    #특정 노드에 대해서 가장 가까운 노드 순서가 되도록 초기화
    def init_nearest_nodes(self):        
        for node in self.nodes:
            node.init_nearest()
        
#    def get_dist(self, start, end):
#        return self.nodes[start].distances[end]

    
    
#노드의 속성
class Node:
    #현재 노드 번호와 각 대상 노드들로의 거리들
    def __init__(self, node_num, distances):
        self.visited = False
        self.node_num = node_num
        self.distances = distances
        self.clustered = False
        self.return_node = False
        self.connected_return_node = False
        
    #현재 노드에서 특정 대상 노드까지의 거리 반환// getter
    def get_distance(self, index):
        return self.distances[index]
        
    #노드 방문 상태 설정// setter
    def set_visited(self, visited):
        self.visited = visited
        
    #노드 방문 여부 확인// True, False 반환, Boolean에서나 사용
    def is_visited(self):
        return self.visited
    
        
    # 현 노드 기준으로 가장 가까운 노드 순서대로 정렬함 (거리, 노드인덱스)
    def init_nearest(self):
        nearest = [(self.distances[idx], idx) for idx in range(len(self.distances))]
        nearest.sort()
        self.nearest = nearest



# DR_1의 속성
class DR_1:
    def __init__(self):    
        self.SK_capa = 3   #DR 1회 가능 방문 수(triple visit)
        self.SK_disable_nodes = [ ]
        self.SK_num = "DR_1"

                
    #노드 방문 상태 설정// setter
    def set_disable_nodes(self, node):
        self.SK_disable_nodes.append(node)
    
    def get_disable_nodes(self):
        return self.SK_disable_nodes
        

        
        
        
        
        
        
# UAV_1의 속성
class UAV_1:
    def __init__(self):    
#        self.SK_num = SK_num
        self.SK_capa = 1   #UAV 1회 가능 방문 수(single visit)
        self.SK_disable_nodes = [ ]
        self.SK_num = "UAV_1"

                
    #노드 방문 상태 설정// setter
    def set_disable_nodes(self, node):
        self.SK_disable_nodes.append(node)
    
    def get_disable_nodes(self):
        return self.SK_disable_nodes

        

'''
class UAV_2:
    def __init__(self):    
#        self.SK_num = SK_num
        self.SK_capa = 1   #UAV 1회 가능 방문 수(single visit)
        self.SK_disable_nodes = [ ]
        self.SK_num = "UAV_2"

                
    #노드 방문 상태 설정// setter
    def set_disable_nodes(self, node):
        self.SK_disable_nodes.append(node)
    
    def get_disable_nodes(self):
        return self.SK_disable_nodes


'''


'\nclass UAV_2:\n    def __init__(self):    \n#        self.SK_num = SK_num\n        self.SK_capa = 1   #UAV 1회 가능 방문 수(single visit)\n        self.SK_disable_nodes = [ ]\n        self.SK_num = "UAV_2"\n\n                \n    #노드 방문 상태 설정// setter\n    def set_disable_nodes(self, node):\n        self.SK_disable_nodes.append(node)\n    \n    def get_disable_nodes(self):\n        return self.SK_disable_nodes\n\n\n'

In [79]:
class Program:

    def __init__(self):
        reader = CSVReader() #CSV 파일 읽어옴
        self.tau_graph = reader.load_csv('tau_v2.csv')
        self.taup_graph = reader.load_csv('taup_v2.csv')
        self.tau_graph.init_nearest_nodes()  #nearest 노드로 정렬 초기화
        self.taup_graph.init_nearest_nodes()
        
        self.SK_node_list_origin = [3, 8]  #이전 시퀀스에서 이미 SK 노드로 설정된 노드들
        self.DR_1_node_list_origin = [3]  
        self.UAV_1_node_list_origin = [8]

#        self.new_graph = reader.load_csv('tau_v2.csv')  # tau_graph가 list가 아니라서 deep copy가 안되므로


    # 트럭 노드 리스트 확보 (사이드 킥 제외)
    def greedy_truck_node_list(self, current_SK_node_list):  # current_SK_node_list = [3, 8, 15]  

        greedy_truck_nodes_list = [node.node_num for node in self.tau_graph.nodes]  # [0, 1, 2, 3, 4, ...]

        for SK_node_num in current_SK_node_list:
            greedy_truck_nodes_list.remove(SK_node_num)

        return greedy_truck_nodes_list  #[ 0, 1, 2, 4, 5, 6, 7, ...]

    
    # 트럭 노드 리스트로 TSP 실행
    def greedy_tsp(self, greedy_truck_nodes_list):

        depot_node = greedy_truck_nodes_list[0]  # depot 0번 노드
        greedy_path = [depot_node]  #경로에 대한 번호 리스트
        visit_node = {0:True}
        node_cnt = len(greedy_truck_nodes_list)
        current_node = depot_node

        for j in range(node_cnt-1):

            min_dist = INFINITE
            next_node = None

            for i in range(node_cnt):

                if greedy_truck_nodes_list[i] in visit_node.keys():
                    continue

                cur_distance = self.tau_graph.get_distance(current_node, greedy_truck_nodes_list[i])

                if cur_distance < min_dist:
                    min_dist = cur_distance
                    next_node = greedy_truck_nodes_list[i]            

            visit_node[next_node] = True
            greedy_path.append(next_node)
            current_node = next_node   # current_node를 출발점 기준으로 고정

        greedy_path.append(0)
        
        return greedy_path                

    
    # 트럭 노드 TSP에 따른 Cum_dist 나열
    def get_truck_cum_dist(self, greedy_path):
        truck_path = greedy_path
        total_dist = 0
        truck_cum_dist = [total_dist]

        for idx in range(len(truck_path)-1):

            node = self.tau_graph.get_node(truck_path[idx])  #노드 뽑아내기
            total_dist += node.get_distance(truck_path[idx+1])  #다음 노드까지 거리 적산
            truck_cum_dist.append(total_dist)  #적산된 거리 나열

        return truck_cum_dist
    
    
    
    '''
    주어진 check_node_list에 대해 순차적으로 bike 사용 여부를 결정하여 최종적인 경로를 구한다.
    '''
    def decision_best_path(self, check_node_list):
        for check_node in check_node_list:
            choose_SK_usage(check_node)
            
            
            
    
    
    
    '''
    매개변수로 전달받은 check_node에 대해서 DR_1, UAV_1 혹은 truck 중 어떤 경로에 포함될 것인지를 판단한다.
    [Pre-condition]
    - DR_1, UAV_1가 각각 방문하기 전 확정된 노드들이 존재한다.
    [Input]
    - check_node: DR_1, UAV_1가 방문할 것인지 판단할 노드
    [Output]
    - check_node가 DR_1, UAV_1 중 어떤 경로에 포함되거나, 모두 포함되지 않을지의 여부
    '''
    def choose_SK_usage(self, check_node):
        '''
        A. check_node가 truck 노드에 포함될 경우 truck의 경로 및 총 거리를 구한다.
            - greedy_path = self.get_greedy_path()
        B. check_node가 DR_1에 포함될 경우 DR_1의 경로 및 총 거리를 구한다.
            - greedy_path = self.get_greedy_path()
            - DR_1_path = self.calculate_DR_1_path([DR_1_node_lsit... + check_node], greedy_path)
        C. check_node가 UAV_1에 포함될 경우 UAV_1의 경로 및 총 거리를 구한다.
            - UAV_1_path = self.calculate_UAV_1_path([UAV_1_node_list... + check_node], greedy_path)
        
        A, B, C 를 비교하여 최종적으로 check_node가 어떤 경로에 포함될 것인지 결정한다.
            1. greedy_path로 truck_dist를 구한다.
            2. bike_a_path로 DR_1_dist를 구한다.
            3. bike_b_path로 UAV_1_dist를 구한다.
            1,2,3 값을 비교하여 check_node를 self.DR_1_node_list, self.UAV_1_node_list 중 하나에 추가하거나 둘 다 추가하지 않는다.
        '''
        
        DR_1_node_list = self.DR_1_node_list_origin[:]  #[3]
        DR_1_node_list.append(check_node)  #[3,15]
        
        UAV_1_node_list = self.UAV_1_node_list_origin[:]  #[8]
        UAV_1_node_list.append(check_node)  #[8,15]
        
        SK_node_list = self.SK_node_list_origin[:]  #[3,8]
        SK_node_list.append(check_node)  #[3,8,15]
        
        
        #A. check_node가 트럭에 있을 때 총 거리
        truck_node_list_when_truck = self.greedy_truck_node_list(self.SK_node_list_origin)
        greedy_path_when_truck = self.greedy_tsp(truck_node_list_when_truck)
        DR_1_pathes_when_truck = self.calculate_DR_1_path(self.DR_1_node_list_origin, greedy_path_when_truck)
        UAV_1_pathes_when_truck = self.calculate_UAV_1_path(self.UAV_1_node_list_origin, greedy_path_when_truck)
        #truck_node_total_dist = self.get_mixed_dist(greedy_path_when_truck, DR_1_pathes_when_truck)
        
                
        
        #B. check_node가 DR_1에 있을 때 총 거리
        truck_node_list_when_DR_1 = self.greedy_truck_node_list(SK_node_list)
        greedy_path_when_DR_1 = self.greedy_tsp(truck_node_list_when_DR_1)
        DR_1_pathes_when_DR_1 = self.calculate_DR_1_path(DR_1_node_list, greedy_path_when_DR_1)
        #DR_1_node_total_dist = self.get_mixed_dist(greedy_path_when_DR_1, DR_1_pathes_when_DR_1)
        
                

        
        #C. check_node가 UAV_1에 있을 때 총 거리
        truck_node_list_when_UAV_1 = self.greedy_truck_node_list(SK_node_list)
        greedy_path_when_UAV_1 = self.greedy_tsp(truck_node_list_when_UAV_1)
        UAV_1_pathes_when_UAV_1 = self.calculate_UAV_1_path(UAV_1_node_list, greedy_path_when_UAV_1)
        #UAV_1_node_total_dist = self.get_mixed_dist(greedy_path_when_UAV_1, UAV_1_pathes_when_UAV_1)
        
        
        
        
        
        return DR_1_node_list, DR_1_pathes_when_DR_1, UAV_1_node_list, UAV_1_pathes_when_UAV_1
    
    
    '''
    [Pre-condition]
    - bike_A가 방문하기로 확정된 노드 목록은 DR_1_node_list_try에 기록되어 있다.
    [Input]
    - DR_1_node_list: bikeA가 방문할 것인지 판단할 노드 목록
    - greedy_path: check_node를 포함한 bike_A 방문 노드가 제외된 트럭 경로
    [Output]
    - bike_a_path
    '''

    
    
    
    #DR_1으로 배정될 때 계산 + 기존 DR_1에 복수개의 배정이 가능할 때도 고려   
    
    def calculate_DR_1_path(self, DR_1_node_list, greedy_path):
        
        
        DR_1_pathes = [ ]
        DR_1_input = DR_1()   #객체 선언
        
        for i in DR_1_node_list:
            
            
            DR_1_path = self.choose_dpt_arv_connected(DR_1_input, i, greedy_path)

            # 경로를 찾지 못했을 경우에는 False 한다
            if DR_1_path[1] == None or DR_1_path[-2] == None:
                DR_1_pathes = False

            else:
                DR_1_pathes.append(DR_1_path)

        
        return DR_1_pathes
    
    
 
    

    
    #UAV_1으로 배정될 때 계산    
    
    def calculate_UAV_1_path(self, UAV_1_node_list, greedy_path):
        
        UAV_1_pathes = [ ]
        UAV_1_input = UAV_1()   #객체 선언
        
        for i in UAV_1_node_list:
            UAV_1_path = self.choose_dpt_arv(UAV_1_input, i, greedy_path)
            
            
            # 경로를 찾지 못했을 경우에는 False 한다
            if UAV_1_path[1] == None or UAV_1_path[3] == None:
                UAV_1_pathes = False
            
            else:
                UAV_1_pathes.append(UAV_1_path)
        
        
        return UAV_1_pathes
    
    
    '''
    [Pre-condition]
    - DR_1에는 disabled_node가 저장된 상태이다.
    [Input]
    - bike_num  # 취소 됨
    - target_node
    - greedy_path
    [Output]
    - (bike_num, dpt, target, arv, bike_dist)
    '''

    
    def choose_dpt_arv(self, SK, target_node, greedy_path):
        '''
        A. greedy_path 중 SK가 target_node로 출발할 dpt_node를 결정한다.
            - dpt_node = get_dpt_node(SK, target_node, greedy_path) 
            : greedy_path 중 disabled 노드를 제외한 target_node에서 가장 가까운 노드를 찾는다.
        B. greedy_path 중 SK가 target_node에서 도착할 arv_node를 결정한다.
            - avaiable_nodes = self.search_available_arv_nodes(sk_num, dpt_node, target_node, greedy_path)
            - arv_node = self.get_arv_node(sk_num, dpt_node, target_node, available_nodes)
        
        C. DR_1로 dpt_node -> target_node -> arv_node 이동 시 전체 경로의 거리를 계산한다.
            
        return (sk_num, dpt_node, target_node, arv_node, bike_dist)
        '''
       
        dpt_node = self.get_dpt_node(SK, target_node, greedy_path)
        available_path = self.search_available_arv_nodes(target_node, dpt_node, SK, greedy_path)
        arv_node, total_SK_dist = self.get_arv_node(target_node, dpt_node, SK, greedy_path, available_path)
        
        SK_num = SK.SK_num
        
        return SK_num, dpt_node, target_node, arv_node, total_SK_dist
    
    
    
    '''
    greedy_path 중 bike_num의 disabled_nodes에 포함되지 않은 node 중 target_node와 가장 가까운 노드를 찾는다.
    [Input]
    - bike_num
    - target_node
    - greedy_path
    [Output]
    - dpt_node
    '''
    def get_dpt_node(self, SK, target_node, greedy_path):
        
        
        dpt_node = None
        min_dist = INFINITE

        for i in range(len(greedy_path)):

            if greedy_path[i] in SK.SK_disable_nodes:
                continue
                    
            cur_distance = self.taup_graph.get_distance(target_node, greedy_path[i])

            if cur_distance < min_dist:
                min_dist = cur_distance
                current_node = greedy_path[i]
        
        dpt_node = current_node
        SK.SK_disable_nodes.append(dpt_node)        
        
        return dpt_node
    
    
    '''
    greedy_truck_path 중 dpt_node를 포함하며, bike의 disabled_nodes를 포함하지 않을 수 있는 arv 후보 목록을 찾는다.
    '''
    def search_available_arv_nodes(self, target_node, dpt_node, SK, greedy_path):

        available_path = [ ]    # second_node로 가능한 노드 리스트
        disabled = SK.SK_disable_nodes[:]
    
        ###### pivot algorithm ######
        start_index = 0
        end_index = len(greedy_path) - 1

        first_index = greedy_path.index(dpt_node)  # dpt_node의 인덱스

        for i in disabled:
            tgt_index = greedy_path.index(i)

            if tgt_index < first_index and tgt_index > start_index:
                start_index = tgt_index

            if tgt_index > first_index and tgt_index < end_index:
                end_index = tgt_index

        #start_node = greedy_path[start_index]
        #end_node = greedy_path[end_index]
    
        for i in greedy_path:
            if greedy_path.index(i) < start_index:
                continue
            if greedy_path.index(i) > end_index:
                continue
            if i == dpt_node:
                continue
            available_path.append(i)
        
        print("disable_nodes:", disabled)
        print("available_arv_path:", available_path)
        
        return available_path
    
    
    '''
    dpt_node -> target_node -> arv_node 가 트럭 노드의 dpt~arv의 경로보다 짧을 수 있는 arv_node를 찾는다.
    '''
    def get_arv_node(self, target_node, dpt_node, SK, greedy_path, available_path):

        arv_node = None
        #cand_node_list = ["cand_node_list:"]
        #truck_dist_list_no = ["truck_dist_list_no:"]
        #bike_dist_list_no = ["bike_dist_list_no:"]
        #truck_dist_list_yes = ["truck_dist_list_yes:"]
        #bike_dist_list_yes = ["bike_dist_list_yes:"]
        #arv_node_list = ["arv_node_list:"]
        
        disabled = SK.SK_disable_nodes[:]
        total_SK_dist = 0
        
        for i in range(len(available_path)):   #available path 수 만큼 시행
        
            min_dist = INFINITE
            
            for node in available_path:   #available_path 에서 가장 짧은 노드 선택

                if node in disabled:
                    continue
                cur_distance = self.taup_graph.get_distance(target_node, node)

                if cur_distance < min_dist:
                    min_dist = cur_distance
                    cand_node = node    #선택하여 cand_node로 선언

            #cand_node_list.append(cand_node)

            ## 거리 따져서 가능여부 결정해야 함, 바이크 배송거리 + 60 < 트럭 배송거리 이어야 함
            SK_dist = 0
            SK_path_trying = [dpt_node, target_node, cand_node]

            for idx in range(len(SK_path_trying)-1):   #bike_a_path_trying의 거리 산정

                cur_node = self.taup_graph.get_node(SK_path_trying[idx])
                SK_dist += cur_node.get_distance(SK_path_trying[idx+1])
            
            #truck_dist 산정
            truck_dist = abs(self.get_truck_cum_dist(greedy_path)[greedy_path.index(cand_node)] - self.get_truck_cum_dist(greedy_path)[greedy_path.index(dpt_node)])
            
            
            if SK_dist + 60 > truck_dist: 
                disabled.append(cand_node)
                #truck_dist_list_no.append(truck_dist)
                #bike_dist_list_no.append(bike_dist)
                continue

            else:
                SK.SK_disable_nodes.append(cand_node)    ############## 바이크 배송이 선택되지 않으면 순환되어야 하는데 그렇지 않음
                disabled.append(cand_node)
                total_SK_dist = SK_dist + 60
                #truck_dist_list_yes.append(truck_dist)
                #bike_dist_list_yes.append(bike_dist)
                
                for interval_node in greedy_path:

                    if greedy_path.index(interval_node) > greedy_path.index(dpt_node) : 

                        if greedy_path.index(interval_node) < greedy_path.index(cand_node):
                            SK.SK_disable_nodes.append(interval_node)
                            disabled.append(interval_node)
                        else:
                            continue
                    else:
                        continue

                    if greedy_path.index(interval_node) < greedy_path.index(dpt_node) : 

                        if greedy_path.index(interval_node) > greedy_path.index(cand_node):
                            SK.SK_disable_nodes.append(interval_node)
                            disabled.append(interval_node)
                        else:
                            continue
                    else:
                        continue

                if greedy_path.index(dpt_node) > greedy_path.index(cand_node):   #순서 맞추기
                    temp = dpt_node
                    dpt_node = cand_node
                    cand_node = temp
                
                #arv_node_list.append(cand_node)
                break
                
        arv_node = cand_node
            
        
        return arv_node, total_SK_dist #, cand_node_list, bike_dist_list_no, truck_dist_list_no, bike_dist_list_yes, truck_dist_list_yes, arv_node_list

    

    
    def choose_dpt_arv_connected(self, SK, target_node, greedy_path):
       
        dpt_node = self.get_dpt_node(SK, target_node, greedy_path)
        available_path = self.search_available_arv_nodes(target_node, dpt_node, SK, greedy_path)
        arv_node, total_SK_dist = self.get_arv_node(target_node, dpt_node, SK, greedy_path, available_path)
        
        SK_num = SK.SK_num
        
        return SK_num, dpt_node, target_node, arv_node, total_SK_dist
        
    
    
    

In [80]:
program = Program()


result_1= program.greedy_truck_node_list([3, 8, 15])
print("truck_node_list:", result_1)

result_2 = program.greedy_tsp(result_1)
print("greedy_path:", result_2)

result_3 = program.get_truck_cum_dist(result_2)
print("cum_dist_greedy_path:",result_3)

'''
result_4, result_4b = program.get_dpt_node(0, 8, result_2)
print(result_4, result_4b)

result_5 = program.search_available_arv_nodes(8, result_4, result_4b, result_2)
print(result_5)

result_6 = program.get_arv_node(8, result_4, result_4b, result_2, result_5)
print(result_6)

#result_7 = program.choose_dpt_arv( 0, 8, result_2)
#print(result_7)
'''

#result_8 = program.calculate_DR_1_path([3, 15, 8], result_2)
#print("bike_a_path(bike_num, dpt_node, target_node, arv_node, bike_dist):", result_8)

result_9 = program.choose_SK_usage(15)
print(result_9)



truck_node_list: [0, 1, 2, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20]
greedy_path: [0, 1, 14, 16, 4, 6, 19, 11, 9, 10, 18, 17, 5, 7, 12, 2, 13, 20, 0]
cum_dist_greedy_path: [0, 184, 251, 431, 475, 514, 558, 569, 651, 771, 783, 939, 1127, 1382, 1669, 1960, 2023, 2080, 2341]
disable_nodes: [9]
available_arv_path: [0, 1, 14, 15, 11, 19, 6, 4, 16, 2, 13, 20, 10, 18, 17, 5, 7, 12, 0]
disable_nodes: [17]
available_arv_path: [0, 1, 14, 15, 11, 19, 6, 4, 16, 2, 13, 20, 9, 10, 18, 5, 7, 12, 0]
disable_nodes: [9]
available_arv_path: [0, 1, 14, 16, 4, 6, 19, 11, 10, 18, 17, 5, 7, 12, 2, 13, 20, 0]
disable_nodes: [9, 18, 10, 14]
available_arv_path: [0, 1, 16, 4, 6, 19, 11, 9, 0]
disable_nodes: [17]
available_arv_path: [0, 1, 14, 16, 4, 6, 19, 11, 9, 10, 18, 5, 7, 12, 2, 13, 20, 0]
disable_nodes: [17, 5, 14]
available_arv_path: [0, 1, 16, 4, 6, 19, 11, 9, 10, 18, 17, 0]
([3, 15], [('DR_1', 9, 3, 18, 128), ('DR_1', 14, 15, 11, 194)], [8, 15], [('UAV_1', 17, 8, 5, 173), ('UAV_1', 14, 15, 

In [65]:
DR_1_pathes = [(1,3),(2)]
print (DR_1_pathes[0][1])

3
